In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get API key:
STEAM_API_KEY = os.getenv("STEAM_API_KEY")




API Key Loaded: 856C9*****


In [2]:
import requests
import os
from dotenv import load_dotenv

# Load API key from .env
load_dotenv()
STEAM_API_KEY = os.getenv("STEAM_API_KEY")

def fetch_steam_data(endpoint, params={}):
    """Fetch data from a Steam API endpoint."""
    base_url = "https://api.steampowered.com/"
    url = base_url + endpoint
    params["key"] = STEAM_API_KEY  # Add API key to parameters
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises an error for bad responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print("Error fetching data:", e)
        return None


In [3]:
def get_player_count(app_id):
    """Fetch the current number of players for a specific game."""
    endpoint = "ISteamUserStats/GetNumberOfCurrentPlayers/v1/"
    params = {"appid": app_id}
    data = fetch_steam_data(endpoint, params)
    
    if data and "response" in data:
        return data["response"].get("player_count", "N/A")
    return "Error fetching player count"

# Example: Get player count for Counter-Strike 2 (app ID: 730)
app_id = 730
player_count = get_player_count(app_id)
print(f"Current players for App {app_id}: {player_count}")


Current players for App 730: 966975


In [7]:
import requests
import os
from dotenv import load_dotenv

# Load the API key from .env file
load_dotenv()
API_KEY = os.getenv("STEAM_API_KEY")

# Define the list of games and their app_ids
games = {
    "Counter-Strike 2": "730",  # app_id for Counter-Strike 2
    "PUBG: BATTLEGROUNDS": "578080",  # app_id for PUBG: BATTLEGROUNDS
    "Dota 2": "570",  # app_id for Dota 2
    "Apex Legends": "1172470",  # app_id for Apex Legends
    "Rust": "252490"  # app_id for Rust
}

# Function to fetch reviews for each game
def fetch_reviews(app_id):
    url = f"https://store.steampowered.com/appreviews/{app_id}?json=1&key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  # Return reviews data
    else:
        print(f"Failed to fetch reviews for app_id: {app_id}")
        return None

# Fetch reviews for each game
reviews_data = {}
for game, app_id in games.items():
    print(f"Fetching reviews for {game}...")
    reviews_data[game] = fetch_reviews(app_id)

# Print the fetched data for verification
for game, data in reviews_data.items():
    if data:
        print(f"Reviews for {game}: {data}")


Fetching reviews for Counter-Strike 2...
Fetching reviews for PUBG: BATTLEGROUNDS...
Fetching reviews for Dota 2...
Fetching reviews for Apex Legends...
Fetching reviews for Rust...
Reviews for Counter-Strike 2: {'success': 1, 'query_summary': {'num_reviews': 20, 'review_score': 8, 'review_score_desc': 'Very Positive', 'total_positive': 1124271, 'total_negative': 176818, 'total_reviews': 1301089}, 'reviews': [{'recommendationid': '190591762', 'author': {'steamid': '76561199352850068', 'num_games_owned': 235, 'num_reviews': 4, 'playtime_forever': 50196, 'playtime_last_two_weeks': 1081, 'playtime_at_review': 49771, 'last_played': 1742814822}, 'language': 'english', 'review': "cs2 is free , but I'm paying my sanity for it\n\n\ncheaters telling me I'm the cheater\ncheaters telling me they turned on their wall hack because they didn't like my mustache\ncheaters act like we don't know they are cheating\ncheaters one tap me and tell me its skill issue\ncheaters pretend they don't use cheats s

In [23]:
import requests
import os
from dotenv import load_dotenv

# Load the API key from .env file
load_dotenv()
API_KEY = os.getenv("STEAM_API_KEY")

# List of selected game IDs (the games you're analyzing)
selected_games = {
    'Counter-Strike 2': '730',  # app_id for Counter-Strike 2
    'PUBG: BATTLEGROUNDS': '578080',  # app_id for PUBG: BATTLEGROUNDS
    'Dota 2': '570',  # app_id for Dota 2
    'Apex Legends': '1172470',  # app_id for Apex Legends
    'Rust': '252490'  # app_id for Rust
}

# API URLs
DETAILS_API_URL = "https://store.steampowered.com/api/appdetails"
REVIEWS_API_URL = "https://store.steampowered.com/appreviews/{app_id}?json=1&key={api_key}&num_per_page=100&page={page}"  # Adding pagination for reviews

# Fetch Game Details (including overall review score and total reviews)
def fetch_game_details(game_id):
    try:
        # Fetch game details (reviews and name)
        response = requests.get(DETAILS_API_URL, params={"appids": game_id})
        data = response.json()

        if str(game_id) in data and 'data' in data[str(game_id)]:
            game_data = data[str(game_id)]['data']
            name = game_data.get('name', 'Unknown')

            # Fetch total reviews and overall review score from the appreviews API (with pagination)
            review_data = fetch_reviews(game_id)
            if review_data:
                total_reviews = review_data['total_reviews']
                total_positive = review_data['total_positive']
                overall_review_score = review_data['review_score_desc']
                review_percentage = (total_positive / total_reviews) * 100 if total_reviews else 0

                # If it's available, also get the player count (via another API)
                player_count = fetch_player_count(game_id)

                return name, total_reviews, review_percentage, overall_review_score, player_count
        return None
    except Exception as e:
        print(f"Error fetching game details for game {game_id}: {e}")
        return None

# Fetch Reviews data with pagination
def fetch_reviews(game_id):
    try:
        total_reviews = 0
        total_positive = 0
        page = 1

        while True:
            reviews_url = REVIEWS_API_URL.format(app_id=game_id, api_key=API_KEY, page=page)
            review_data = requests.get(reviews_url).json()

            if 'query_summary' in review_data:
                total_reviews += review_data['query_summary'].get('total_reviews', 0)
                total_positive += review_data['query_summary'].get('total_positive', 0)
                if 'total_reviews' in review_data['query_summary']:
                    break
            page += 1
            if page > 10:  # Limit to 10 pages to avoid endless loop (adjust as needed)
                break

        if total_reviews > 0:
            review_score_desc = review_data['query_summary'].get('review_score_desc', 'N/A')
            return {
                'total_reviews': total_reviews,
                'total_positive': total_positive,
                'review_score_desc': review_score_desc
            }

        return None
    except Exception as e:
        print(f"Error fetching reviews for game {game_id}: {e}")
        return None

# Fetch Player Count from Steam API
def fetch_player_count(game_id):
    try:
        # Use a different API endpoint for player count
        response = requests.get(f'https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/', params={'appid': game_id})
        data = response.json()

        if 'response' in data:
            return data['response'].get('player_count', 'N/A')
        return 'N/A'
    except Exception as e:
        print(f"Error fetching player count for game {game_id}: {e}")
        return 'N/A'

# Function to fetch all selected games' details
def fetch_selected_games_data():
    games_data = []

    for game_name, game_id in selected_games.items():
        game_data = fetch_game_details(game_id)
        if game_data:
            games_data.append(game_data)
    
    return games_data

# Fetch data for selected games
games_info = fetch_selected_games_data()

# Print the results
for game in games_info:
    print(f"Game: {game[0]} - Total Reviews: {game[1]} - Review Percentage: {game[2]:.2f}% - Overall Review Score: {game[3]} - Player Count: {game[4]}")


Game: Counter-Strike 2 - Total Reviews: 1301089 - Review Percentage: 86.41% - Overall Review Score: Very Positive - Player Count: 1011366
Game: PUBG: BATTLEGROUNDS - Total Reviews: 276645 - Review Percentage: 63.20% - Overall Review Score: Mixed - Player Count: 358554
Game: Dota 2 - Total Reviews: 6092 - Review Percentage: 89.00% - Overall Review Score: Very Positive - Player Count: 314556
Game: Apex Legends™ - Total Reviews: 1015 - Review Percentage: 80.10% - Overall Review Score: Very Positive - Player Count: 92131
Game: Rust - Total Reviews: 483535 - Review Percentage: 87.13% - Overall Review Score: Very Positive - Player Count: 78212


In [33]:
import requests
import os
from dotenv import load_dotenv

# Load the API key from .env file
load_dotenv()
API_KEY = os.getenv("STEAM_API_KEY")

# List of selected game IDs (the games you're analyzing)
selected_games = {
    'Counter-Strike 2': '730',  # app_id for Counter-Strike 2
    'PUBG: BATTLEGROUNDS': '578080',  # app_id for PUBG: BATTLEGROUNDS
    'Dota 2': '570',  # app_id for Dota 2
    'Apex Legends': '1172470',  # app_id for Apex Legends
    'Rust': '252490'  # app_id for Rust
}

# API URL for reviews (without pagination)
REVIEWS_API_URL = "https://store.steampowered.com/appreviews/{app_id}?json=1&key={api_key}"  # Summary data only (no pagination)

# Fetch Reviews data (summary only)
def fetch_reviews(game_id):
    try:
        reviews_url = REVIEWS_API_URL.format(app_id=game_id, api_key=API_KEY)
        review_data = requests.get(reviews_url).json()

        if 'query_summary' in review_data:
            total_reviews = review_data['query_summary'].get('total_reviews', 0)
            total_positive = review_data['query_summary'].get('total_positive', 0)
            review_score_desc = review_data['query_summary'].get('review_score_desc', 'N/A')

            # Calculate Review Percentage
            review_percentage = (total_positive / total_reviews) * 100 if total_reviews else 0

            return {
                'total_reviews': total_reviews,
                'total_positive': total_positive,
                'review_score_desc': review_score_desc,
                'review_percentage': review_percentage
            }
        return None
    except Exception as e:
        print(f"Error fetching reviews for game {game_id}: {e}")
        return None

# Fetch Player Count from Steam API
def fetch_player_count(game_id):
    try:
        # Use a different API endpoint for current player count
        response = requests.get(f'https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/', params={'appid': game_id})
        data = response.json()

        if 'response' in data:
            return data['response'].get('player_count', 'N/A')
        return 'N/A'
    except Exception as e:
        print(f"Error fetching player count for game {game_id}: {e}")
        return 'N/A'

# Fetch Game Details and Reviews (including total reviews, review percentage, and player count)
def fetch_game_details(game_id, game_name):
    try:
        review_data = fetch_reviews(game_id)
        if review_data:
            total_reviews = review_data['total_reviews']
            total_positive = review_data['total_positive']
            review_percentage = review_data['review_percentage']
            overall_review_score = review_data['review_score_desc']
            player_count = fetch_player_count(game_id)

            return game_name, total_reviews, review_percentage, overall_review_score, player_count
        return None
    except Exception as e:
        print(f"Error fetching game details for {game_name}: {e}")
        return None

# Function to fetch all selected games' details
def fetch_selected_games_data():
    games_data = []

    for game_name, game_id in selected_games.items():
        game_data = fetch_game_details(game_id, game_name)
        if game_data:
            games_data.append(game_data)
    
    return games_data

# Fetch data for selected games
games_info = fetch_selected_games_data()

# Print the results
for game in games_info:
    print(f"Game: {game[0]} - Total Reviews: {game[1]} - Review Percentage: {game[2]:.2f}% - Overall Review Score: {game[3]} - Player Count: {game[4]}")


Game: Counter-Strike 2 - Total Reviews: 1301093 - Review Percentage: 86.41% - Overall Review Score: Very Positive - Player Count: 1029697
Game: PUBG: BATTLEGROUNDS - Total Reviews: 276645 - Review Percentage: 63.20% - Overall Review Score: Mixed - Player Count: 351835
Game: Dota 2 - Total Reviews: 6092 - Review Percentage: 89.00% - Overall Review Score: Very Positive - Player Count: 330960
Game: Apex Legends - Total Reviews: 1015 - Review Percentage: 80.10% - Overall Review Score: Very Positive - Player Count: 93026
Game: Rust - Total Reviews: 483535 - Review Percentage: 87.13% - Overall Review Score: Very Positive - Player Count: 81479
